In [225]:
import os, sys, warnings
import concurrent.futures
from openpyxl import Workbook
from multiprocessing import Pool
from pathlib import Path
from IPython.display import Image, display
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
#sys.path.insert(0, os.path.abspath(os.path.join(Path.home(), "rpa_libs")))

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys

# Nossas libs
#import app
#from plugins.selenium.selenium import Selenium

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [226]:
# Exemplo de uso do arquivo de configuração
#app.redefine_config('caminho do seu arquivo .yaml')
#teste
import pandas as pd

In [227]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [228]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [229]:
# Def the website i want to extract the table for
url = 'https://www.coingecko.com/pt'
driver.get(url)

In [230]:
# Picking table from Cripto Coins site and creating a variable for the rows in this table
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [231]:
def maybe_float(s):
    try:
        return round(float(s.replace(".", "").replace(",", ".")), 2)
    except (ValueError, TypeError):
        return s

In [232]:
# Creating the function to iterate in rows and cells of the table and appending to the table_data
def process_cells(cells):
    cell_data = []
    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        data_sort = cell.get_attribute('data-sort')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)

        if '$' in cell.text:
            cell_splited = cell.text.split(" ")
            for part_splitted in cell_splited:  
                cell_data.append(maybe_float(part_splitted))
        elif data_sort:
            cell_data.append(round(float(data_sort), 2))
        else:
            cell_data.append(cell.text)
    #table_data.append(cell_data)
    return cell_data

In [233]:
x =22323.24
isinstance(x, float)

True

In [234]:
#with concurrent.futures.ThreadPoolExecutor() as executor:
#    results = list(executor.map(process_rows, rows))

#with Pool() as pool:
#    result = pool.map(process_rows, rows)

In [235]:
def process_rows(rows):
    table_data = []
    for row in rows[:3]:
    #for row in rows[:10]:
        cells = row.find_elements(By.XPATH, './/td')
        cells_data = process_cells(cells)
        table_data.append(cells_data)
    return table_data

In [236]:
table_data = process_rows(rows)

In [237]:
print(table_data[0][7])
print(type(table_data[0][7]))
list(table_data)

0.14
<class 'float'>


[['',
  '1',
  'https://assets.coingecko.com/coins/images/1/standard/bitcoin.png?1696501400',
  'Bitcoin\nBTC',
  'Comprar',
  'US$',
  63462.23,
  0.14,
  -1.27,
  5.67,
  0.03,
  'US$',
  24209258294.0,
  'US$',
  1253944348471.0,
  1332980202814.24,
  '',
  'https://www.coingecko.com/coins/1/sparkline.svg',
  ''],
 ['',
  '2',
  'https://assets.coingecko.com/coins/images/279/standard/ethereum.png?1696501628',
  'Ethereum\nETH',
  'Comprar',
  'US$',
  2578.27,
  -0.11,
  -2.85,
  11.49,
  -3.81,
  'US$',
  13912352314.0,
  'US$',
  310461495306.0,
  310542861061.97,
  '',
  'https://www.coingecko.com/coins/279/sparkline.svg',
  ''],
 ['',
  '3',
  'https://assets.coingecko.com/coins/images/325/standard/Tether.png?1696501661',
  'Tether\nUSDT',
  'Comprar',
  'US$',
  1.0,
  -0.1,
  -0.03,
  -0.17,
  0.01,
  'US$',
  45102913027.0,
  'US$',
  119211346066.0,
  119223661833.52,
  '',
  'https://www.coingecko.com/coins/325/sparkline.svg',
  '']]

In [238]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]
headers[0] = "IMG Coin"
headers.insert(0, "ID")
headers.insert(3, "$")
headers.insert(8, "30 d")
headers.insert(9, "$")
headers.insert(11, "$")
headers.insert(13, "FDV")
#table_data = [[cell for cell in row if cell.strip()] for row in table_data]

In [239]:
list(headers)

['ID',
 'IMG Coin',
 'Moeda',
 '$',
 'Preço',
 '1 h',
 '24 h',
 '7 d',
 '30 d',
 '$',
 'Volume em 24 h',
 '$',
 'Capitalização de mercado',
 'FDV',
 'Últimos 7 dias']

In [240]:
#list(table_data)

In [241]:
df = pd.DataFrame(table_data)
df_treated_with_link = df.loc[:, (df != '').any(axis=0)] #test drop_na
df_treated_with_link = df_treated_with_link.drop(df.columns[4], axis=1)


In [243]:
df_treated_with_link.style\
    .highlight_max(subset=[5], color='green')\
    .background_gradient(cmap='viridis', subset=[4])\
    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})
df_treated_with_link = df_treated_with_link.reset_index(drop=True)

In [247]:
df_treated_with_link.columns = [headers]
df_treated_with_link

,ID,IMG Coin,Moeda,$,Preço,1 h,24 h,7 d,30 d,$,Volume em 24 h,$,Capitalização de mercado,FDV,Últimos 7 dias
0,1,https://assets.coingecko.com/coins/images/1/st...,Bitcoin\nBTC,US$,63462.23,0.14,-1.27,5.67,0.03,US$,2.420926e+10,US$,1.253944e+12,1.332980e+12,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,Ethereum\nETH,US$,2578.27,-0.11,-2.85,11.49,-3.81,US$,1.391235e+10,US$,3.104615e+11,3.105429e+11,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,Tether\nUSDT,US$,1.00,-0.10,-0.03,-0.17,0.01,US$,4.510291e+10,US$,1.192113e+11,1.192237e+11,https://www.coingecko.com/coins/325/sparkline.svg


In [248]:
df_treated = df_treated_with_link.drop(columns=['IMG Coin', 'Últimos 7 dias'])

In [ ]:
df_treated

In [246]:
df_treated.to_excel("file_test_df_cripto_coin.xlsx", index=False)